In [1]:
import regex as re
import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from time import sleep

In [11]:
my_options = webdriver.ChromeOptions()
my_options.add_argument('--disable-extensions')                     # 禁用瀏覽器擴展，減少干擾
my_options.add_argument('--disable-browser-side-navigation')        # 禁用瀏覽器側導航，提高穩定性
my_options.add_argument("--window-size=720,540")                  # 設定瀏覽器視窗大小
my_options.add_argument("--incognito")               # 開啟無痕模式
my_options.add_argument("--disable-popup-blocking")  # 禁用彈出攔截
my_options.add_argument("--disable-notifications")   # 取消 chrome 推播通知
my_options.add_argument("--lang=zh-TW")              # 設定為正體中文
# my_options.add_argument('--no-sandbox')                             # 禁用沙盒模式，提高在某些環境下的穩定性
# my_options.add_argument('--disable-dev-shm-usage')                  # 禁用/dev/shm使用，避免內存不足問題
# my_options.add_argument('--disable-gpu')                            # 禁用GPU加速，避免某些圖形相關問題
# my_options.add_argument('--disable-infobars')                       # 禁用信息欄，如"Chrome正在被自動化軟件控制"的提示
# my_options.add_argument('--disable-features=VizDisplayCompositor')  # 禁用視覺顯示合成器，解決某些渲染問題

folderPath = 'project_gutenberg'
if not os.path.exists(folderPath):
    os.makedirs(folderPath)

In [36]:
def visit(url):
    global driver
    driver = webdriver.Chrome(
    options=my_options
    )
    driver.get(url)
    sleep(3)

def parse():
    book = {}
    elements = driver.find_elements(
        By.CSS_SELECTOR, 'ul li.pgdbetext a[href]'
    )
    # 使用列表推導式提取每個元素的 href 屬性
    # return [element.get_attribute('href') for element in elements]
    print(f"找到 {len(elements)} 個連結")
    for i in elements[:2]:
        try:
            title = i.text.strip()
            href_index = re.findall(r"\d+", i.get_attribute('href'))[0]
            href = f"https://www.gutenberg.org/cache/epub/{href_index}/pg{href_index}.txt"
            print(title)
            print(href)
            book.setdefault(title, href)
            return book
        except (AttributeError, IndexError):
            print("無法解析連結或標題")
            continue

def savetxt(title, content):
    if not os.path.exists(folderPath):
        os.makedirs(folderPath)
    with open(f"{folderPath}/{title}.txt", "w", encoding='utf-8') as file:
        if file.name in os.listdir(folderPath):
            print(f"{title}.txt 已存在，跳過保存")
        else:
            file.write(content)



def exit():
    global driver
    try:
        driver.quit()
    except Exception as e:
        print(f"退出瀏覽器時發生錯誤: {e}")
    finally:
        print("瀏覽器已關閉")

In [37]:
visit('https://www.gutenberg.org/browse/languages/zh')
parse()
exit()

找到 507 個連結
豆棚閒話
https://www.gutenberg.org/cache/epub/25328/pg25328.txt
瀏覽器已關閉
